In [33]:
import os
from scipy import ndimage, misc
import numpy as np
import re
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
def load_images(path):
    images = []
    count = 0
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                filepath = os.path.join(root, filename)
                image = plt.imread(filepath)
                image_resized = cv2.resize(image, (299, 299))
                images.append(image_resized)
                #print count
            #count += 1 
    images = np.asarray(images)
    return images

In [3]:
train = load_images("train")
test = load_images("test")

In [6]:
train.shape

(10222, 256, 256, 3)

In [7]:
test.shape

(10357, 256, 256, 3)

In [14]:
labels = pd.read_csv('labels.csv')

In [15]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [17]:
target = pd.get_dummies(labels.breed, columns=['breed'], drop_first=False)

In [34]:
target.shape

(10222, 120)

In [22]:
X,X_cv,y,y_cv = train_test_split(train,target,test_size = 0.2, random_state = 4)

In [23]:
X.shape

(8177, 256, 256, 3)

In [24]:
y.shape

(8177, 120)

In [32]:

import tensorflow as tf
import pickle as cPickle

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

In [26]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

In [27]:
# Network Parameters
num_input = 268203 
num_classes = 120 
dropout = 0.25 

In [35]:
###########################################################

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # CIFAR-100 images are 32x32 pixels, and have RGB color channel
  input_layer = tf.convert_to_tensor(features)

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 32, 32, 3]
  # Output Tensor Shape: [batch_size, 32, 32, 32]
  # NOTE: tf.layers.conv2d input MUST be floating point for some reason???
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  
  conv2 = tf.layers.conv2d(
      inputs= conv1,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  
  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 32, 32, 32]
  # Output Tensor Shape: [batch_size, 16, 16, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  
  # Add dropout operation; 0.75 probability that element will be kept
  dropout1 = tf.layers.dropout(
      inputs=pool1, rate=0.25, training=mode == learn.ModeKeys.TRAIN)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 16, 16, 32]
  # Output Tensor Shape: [batch_size, 16, 16, 64]
  conv3 = tf.layers.conv2d(
      inputs=dropout1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  conv4 = tf.layers.conv2d(
      inputs= conv3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 16, 16, 64]
  # Output Tensor Shape: [batch_size, 8, 8, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
  
  # Add dropout operation; 0.75 probability that element will be kept
  dropout2 = tf.layers.dropout(
      inputs=pool2, rate=0.25, training=mode == learn.ModeKeys.TRAIN)


  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 8, 8, 64]
  # Output Tensor Shape: [batch_size, 4096]
  pool2_flat = tf.reshape(dropout2, [-1, 8 * 8 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 4096]
  # Output Tensor Shape: [batch_size, 1024]
  # dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  
  # Add dropout operation; 0.6 probability that element will be kept
  dropout3 = tf.layers.dropout(
      inputs=dense1, rate=0.4, training=mode == learn.ModeKeys.TRAIN)
  
  # second fully connected layer = use 512 features
  dense2 = tf.layers.dense(inputs=dropout3, units=512, activation=tf.nn.relu)
  
   # Add dropout operation; 0.6 probability that element will be kept
  dropout4 = tf.layers.dropout(
      inputs=dense2, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 512]
  # Output Tensor Shape: [batch_size, 100] since CIFAR-100 has 100 possible classes
  logits = tf.layers.dense(inputs=dropout4, units=100)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  # !!! Make sure depth variable is set properly relative to data !!!!
  # For CIFAR-100 there are 100 possible classifications, so depth = 100
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=100)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.0001,
        optimizer="Adam")

  # Generate Predictions
  predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [36]:
########################################################################
# CIFAR-100 data loader

def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo, encoding='latin1')
    fo.close()
    return dict

#########################################################################

In [38]:
def main(unused_argv):
  # unpickle CIFAR-100 data from local directory
  d = unpickle('C:/SQLData/661/CIFAR100/train')
  # now extract image data from unpickled object. Result will be 50000 x 3072 array
  # wherein each row is 3072 unraveled image

  data = d['data']
  data = data.astype('float32')
  train_data = np.transpose(np.reshape(data,(-1,32,32,3), order='F'),axes=(0,2,1,3)) #order batch,x,y,color
  train_labels = np.asarray(d['fine_labels'], dtype=np.int32)
    
  # cleanup memory
  del d
  del data
  
  # Create the Estimator
  cifar_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/cifar100v2")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=100)
  
  # Train the model
  with tf.device('/cpu:0'):
      cifar_classifier.fit(
              x=train_data,
              y=train_labels,
              batch_size=64,
              steps=5240,
              monitors=[logging_hook])

  # Configure the accuracy metric for evaluation
  metrics = {
      "accuracy":
          # learn.metric_spec.MetricSpec(
          learn.MetricSpec(
              metric_fn=tf.metrics.accuracy, prediction_key="classes"),
  }

    # eval small subset of training data
  small_train =  np.array_split(train_data, 50)[7]
  small_labs =  np.array_split(train_labels, 50)[7]
  eval_results = cifar_classifier.evaluate(
      x= small_train, y=small_labs, metrics=metrics)
  print(eval_results)

  # remove training data from memory
  del train_data
  del train_labels
   
  ################## EVAL #############################
  # now load eval data
  
  d = unpickle('C:/SQLData/661/CIFAR100/test')
  # now extract testing data from unpickled object. Result will be 10000 x 3072 array
  # wherein each row is 3072 unraveled image

  eval_data = d['data']
  eval_data = eval_data.astype('float32')
  eval_data = np.transpose(np.reshape(eval_data,(-1,32,32,3), order='F'),axes=(0,2,1,3))
  eval_labels = np.asarray(d['fine_labels'], dtype=np.int32)
  
  del d
  
  # Evaluate all test data and print results
  acc = 0
  e_loss = 0
  n_batches = 20
  for i in range (0, n_batches - 1):
    small_eval =  np.array_split(eval_data, n_batches)[i]
    small_labs =  np.array_split(eval_labels, n_batches)[i]
    eval_results = cifar_classifier.evaluate(
      x=small_eval, y=small_labs, metrics=metrics)
    print(eval_results)
    acc += eval_results["accuracy"]
    e_loss += eval_results["loss"]
  
  overall_acc = acc / n_batches
  overall_loss = e_loss/ n_batches
  print("Eval Accuracy = ", overall_acc)
  print("Eval Loss = ", overall_loss)

In [40]:
if __name__ == "__main__":
  tf.app.run()

IOError: [Errno 2] No such file or directory: 'C:/SQLData/661/CIFAR100/train'